In [ ]:
# Import packages
import numpy as np
import pandas as pd
import re

In [ ]:
# Read in data
df = pd.read_json("data/raw_data.json")
df.head()

As of now, the ingridients are lists of strings. Ingredient name, quantity and units are merged together. We need to split them into separate columns.


In [ ]:
class RecipeIngredient:
    name = ""
    quantity = 0
    unit = ""

    def __init__(self, name, quantity, unit):
        self.name = name
        self.quantity = quantity
        self.unit = unit

    def __repr__(self):
        if self.quantity:
            return f"<{self.name} | {self.quantity} {self.unit}>"

        return f"{self.name} | {self.unit}"

    def __str__(self):
        return self.__repr__()

In [ ]:
# The supposed format of each string is:
# ingredient_name - quantity units

# We will split by \U+02013 or -

def process_ingredient_list(ingredient_list):
    processed_ingredients = []

    for ingredient_line in ingredient_list:

        if type(ingredient_line) != str:
            processed_ingredients.append(ingredient_line)

        splitted = ingredient_line.split("–")

        if (len(splitted) != 2):
            continue

        ingredient_name = splitted[0].strip()

        # splitted[1] = " ".join(re.findall(r'[\w.,]+', splitted[1]))

        quantity_units = re.split(r'(\d+,?\d*)', splitted[1].strip())
        quantity_units = list(map(lambda x: x.strip(), quantity_units))
        quantity_units = list(filter(None, quantity_units))

        match len(quantity_units):
            case 1:
                processed_ingredients.append(
                    RecipeIngredient(ingredient_name, None, quantity_units[0])
                )

            case 2:
                processed_ingredients.append(
                    RecipeIngredient(
                        ingredient_name, quantity_units[0].replace(",", "."), quantity_units[1])
                )

            case _:
                print(ingredient_line)
                # raise Exception(
                #     "Unexpected number of elements in quantity_units")

    return processed_ingredients

In [ ]:
# Test process_ingredient_list function
process_ingredient_list(df["ingredients"][0])

In [ ]:
# Apply to the whole dataframe
df["ingredients"] = df["ingredients"].apply(process_ingredient_list)